<a href="https://colab.research.google.com/github/hrbjoern/2024-09_CucumberIntelligence/blob/main/notebooks/Finetuning_YOLO_with_labeltype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning a pretrained YOLO detection model with bounding boxes OR segmentations:

## Imports:

In [ ]:
import os
from datetime import datetime
import pytz
#import tensorflow as tf

## Some config:

In [ ]:
setupnumber = '_alle' # ONLY for saving the model! <-- TODO: cp noch pythonisieren? :-)

modeltype = "yolov8n.pt"
# Other models:
# "yolov8m-seg.pt"
# "yolov8s.pt"

# Train on bounding boxes or segmentations?
#labeltype = "boxes"
labeltype = 'segs'
if labeltype == 'segs':
    modeltype = modeltype.replace(".pt", "-seg.pt")

# Bigger batchsizes for smaller models:
batchsize = 10
if modeltype == "yolov8s.pt":
    batchsize = 40
if modeltype == "yolov8n.pt":
    batchsize = 120
if modeltype == "yolov8n-seg.pt":
    batchsize = 100

imagesize = 640 # 1280
if imagesize == 1280:
    batchsize = batchsize // 4

N_epochs = 300

localtime = datetime.now(pytz.timezone('Europe/Berlin'))
starttime = localtime.strftime("%Y-%m-%d_%H%M")
savestring = "Training date = "+starttime
savestring += "\nSetup = "+setupnumber
savestring += "\nModel = "+modeltype
savestring += "\nLabeltype = "+labeltype
savestring += "\nBatchsize = "+ str(batchsize)
savestring += "\nImagesize = "+ str(imagesize)
savestring += "\nEpochs = " + str(N_epochs)
print(savestring)


Training date = 2024-09-28_2146
Setup = _alle
Model = yolov8n-seg.pt
Labeltype = segs
Batchsize = 120
Imagesize = 640
Epochs = 300


## Fetch the data:

In [ ]:
# Mount Google Drive and copy data:
if 'data_loaded' not in globals():
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir -v /content/Data

    !cp -r /content/drive/MyDrive/setup_015/* /content/Data/ #| pv
    print("setup015 copied")
    !cp -r /content/drive/MyDrive/setup_002/* /content/Data/
    print("setup002 copied")
    !cp -r /content/drive/MyDrive/setup_003/* /content/Data/
    print("setup003 copied")
    !cp -r /content/drive/MyDrive/setup_006/* /content/Data/
    print("setup006 copied")
    !cp -v /content/drive/MyDrive/setup-002-standalone.yaml /content/Data/

    print("Data download finished at", datetime.now(pytz.timezone('Europe/Berlin')).strftime("%H:%M"))


Mounted at /content/drive
mkdir: created directory '/content/Data'
setup015 copied
setup002 copied
setup003 copied
setup006 copied
'/content/drive/MyDrive/setup-002-standalone.yaml' -> '/content/Data/setup-002-standalone.yaml'
Data download finished at 22:01


## Fetch helper scripts/functions and restructure the data:

In [ ]:
if 'data_loaded' not in globals():
    !cp -v /content/drive/MyDrive/prepare_yolo_data_folders.py /content/
    # Download helper script from Github: -> doesn't work.
    #!wget https://raw.githubusercontent.com/<username>/<reponame>/<branch>/prepare_yolo_data_folders.py
    from prepare_yolo_data_folders import prepare_yolo_data_folders

    # Colab magic to make "Data" the current directory:
    %cd /content/Data

    #Do the train-validation split and move the image and label files to YOLO folder structure:
    data_loaded = prepare_yolo_data_folders(labeltype) # TODO: Quiet! :-)
    print('data_loaded is', data_loaded)

'/content/drive/MyDrive/prepare_yolo_data_folders.py' -> '/content/prepare_yolo_data_folders.py'
/content/Data
Preparing files and folders for training with segs labels.
Current folder is: /content/Data
Found 354 labels.
Found 354 labelled images.
Created train/images and train/labels.
Created val/images and val/labels.
Created test/images and test/labels.
Moved 247 images and labels to folders.
Moved 71 images and labels to folders.
Moved 36 images and labels to folders.
Data handling finished.
data_loaded is True


In [ ]:
# Count the files:
num_train = len(os.listdir('/content/Data/train/labels'))
num_val = len(os.listdir('/content/Data/val/labels'))
num_test = len(os.listdir('/content/Data/test/labels'))
print(f"Number of training images: {num_train}")
print(f"Number of validation images: {num_val}")
print(f"Number of test images: {num_test}")

savestring += "\nNumber of training images: " + str(num_train)
savestring += "\nNumber of validation images: " + str(num_val)
savestring += "\nNumber of test images: " + str(num_test)

Number of training images: 247
Number of validation images: 71
Number of test images: 36


In [ ]:
# Create model directory on Drive:
modeldir = '/content/drive/MyDrive/CucumberModels/'+starttime+"_"+labeltype
if not os.path.exists(modeldir):
    os.makedirs(modeldir)

savestring += "\nModel directory: " + modeldir

### Write settings file:

In [ ]:
# Output settings file:
settingsfile = modeldir + "_settings.txt"
with open(settingsfile, "w") as file:
    file.write(savestring)
print("Model settings written to", settingsfile)

Model settings written to /content/drive/MyDrive/CucumberModels/2024-09-28_2146_settings.txt


## Install ultralytics and import YOLO:

In [ ]:
%pip -q install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.1/875.1 kB 49.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Okay, so the images and labels are there? Next: Import YOLO model for training!
Note: Check Bboxes vs. segmentations!

## Load tensorboard:

In [ ]:
%load_ext tensorboard

## Import YOLO model for training

In [ ]:
# Load a pretrained model:
model = YOLO(modeltype)

100%|██████████| 6.74M/6.74M [00:00<00:00, 379MB/s]


### YAML file for folder and class definition

In [ ]:
#!cat /content/misc/setup-002-standalone.yaml
yamlfile = "/content/Data/setup-002-standalone.yaml"

## Do the training!

In [ ]:
# If GPU-RAM is full before training:

# import torch
# torch.cuda.empty_cache()


In [ ]:
# Train the model
trainresults = model.train(data=yamlfile, epochs=N_epochs, imgsz=imagesize, batch=batchsize)

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/Data/setup-002-standalone.yaml, epochs=300, time=None, patience=100, batch=120, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

100%|██████████| 755k/755k [00:00<00:00, 97.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 95.9MB/s]


AMP: checks passed ✅


...

    300/300      19.9G     0.4524     0.6772     0.2819      0.821         57        640: 100%|██████████| 3/3 [00:08<00:00,  2.91s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]

                   all         71        640      0.966      0.961      0.981      0.839      0.966      0.961      0.979      0.793



300 epochs completed in 1.116 hours.
Optimizer stripped from runs/segment/train/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train/weights/best.pt, 6.8MB

Validating runs/segment/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.36s/it]


                   all         71        640      0.966      0.963      0.982      0.844      0.964      0.961      0.979      0.797
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/segment/train


### Save the model directly to Drive:

In [ ]:
model.save(modeldir+"/"+starttime+"_"+labeltype+".pt")

### wandb?

... and managing the parameter "matrix"

### Tensorboard:

In [ ]:
# Show tensorboard:
#%tensorboard --logdir /content/runs/detect #path/to/runs

## Predict the cucumbers on test images:

In [ ]:
# Examples:
# #predictresults = model(["image1.jpg", "image2.jpg"])  # return a list of Results objects
# # Run inference on 'bus.jpg' with arguments
# model.predict("bus.jpg", save=True, imgsz=320, conf=0.5)


# /content/Data/test/images

# First, I guess we should _dynamically_?! define a list of images to predict on:
#imagelist = []
imagelist = ['test/images/'+file for file in os.listdir("/content/Data/test/images")]
print(f"Using {len(imagelist)} images for prediction: ")
print(imagelist)

# Then, we can predict on the images:
results = model.predict(imagelist,
                        imgsz=imagesize,
                        conf=0.6,
                        #show=True,
                        save=True)

for index, result in enumerate(results):
  result.save(modeldir+"/"+starttime+"_prediction_"+str(index)+".jpg")

# # Process result:
# # for result in results:
# boxes = result.boxes  # Boxes object for bounding box outputs
# # masks = result.masks  # Masks object for segmentation masks outputs
# # keypoints = result.keypoints  # Keypoints object for pose outputs
# probs = result.probs  # Probs object for classification outputs
# obb = result.obb  # Oriented boxes object for OBB outputs
# result.show()  # display to screen

Using 36 images for prediction: 
['test/images/dsc01853.png', 'test/images/dsc01868.png', 'test/images/20220614_131249.png', 'test/images/dsc02013.png', 'test/images/dsc01946.png', 'test/images/dsc01827.png', 'test/images/dsc01965.png', 'test/images/dsc01907.png', 'test/images/dsc01932.png', 'test/images/dsc01940.png', 'test/images/dsc01949.png', 'test/images/20220614_131119.png', 'test/images/dsc01996.png', 'test/images/dsc01874.png', 'test/images/20220614_131127.png', 'test/images/dsc01912.png', 'test/images/20220614_131241.png', 'test/images/dsc01989.png', 'test/images/dsc01878.png', 'test/images/dsc02027.png', 'test/images/046.png', 'test/images/20220614_131106.png', 'test/images/016.png', 'test/images/040.png', 'test/images/20220614_131153.png', 'test/images/057.png', 'test/images/dsc01903.png', 'test/images/dsc01845.png', 'test/images/027.png', 'test/images/dsc01928.png', 'test/images/dsc01974.png', 'test/images/dsc01951.png', 'test/images/20220614_131129.png', 'test/images/dsc02

### Show resulting pictures:

Perhaps replace IPython display by PIL stuff?

In [ ]:
#from IPython.display import Image
#Image(filename='/content/runs/detect/train53/20220614_134218_Gurkenfoto_2.jpg', width=400)


In [ ]:
#Image(filename='/content/runs/detect/train2/20220614_134218_Gurkenfoto_rescaled_640.jpg', width=400)


In [ ]:
#Image(filename='/content/runs/detect/train53/dsc01843.png', width=400)